In [3]:
#We have already scrapped the data for topwear,bottomwear & footwear.
#Now I also want to add two more products:- handbags and eyewear
#So lets scrape the data for this two products also

import selenium
import pandas as pd
import time, re
import warnings
warnings.simplefilter("ignore")

from skimage import io
from tqdm import tqdm
from selenium import webdriver as wb
%matplotlib inline



import chromedriver_autoinstaller


chromedriver_autoinstaller.install()  # Check if the current version of chromedriver exists
                                      # and if it doesn't exist, download it automatically,
                                      # then add chromedriver to path

wd = wb.Chrome()

remaining_search_terms = [
               "Men Eyewear Sunglasses",
               "Women Eyewear Sunglasses",
               "Handbags Handbags Handbags"  #The reason i am not putting men handbags & women handbags is bcz it is giving very few results on myntra for these keywords
               ]                             #but keyword 'handbags' are giving lot of results. So I am putting Handbags here and not specifying the gender or type

In [4]:
from selenium.webdriver.common.by import By
from selenium.common.exceptions import WebDriverException, NoSuchElementException, TimeoutException

product_ids, product_urls, image_urls, descriptions, brands, genders, categories, occasions = list(), list(), list(), list(), list(), list(), list(), list()
MAX_PRODUCTS_PER_CATEGORY = 700
for search_term in remaining_search_terms:
    search_term = search_term.lower()
    query = f"https://www.myntra.com/{search_term.replace(' ', '-')}"
    total = 0
    
    retry_count = 0  # Initialize retry count
    max_retries = 3  # Maximum number of retries

    try:
        wd.get(query)
        next_button_found = True

        while total < MAX_PRODUCTS_PER_CATEGORY:
            time.sleep(3)

            try:
                # Use the updated find_elements method with By locator
                products = wd.find_elements(By.CLASS_NAME, "product-base")

                for product in products:
                    try:
                        # Find the product image
                        image = product.find_element(By.TAG_NAME, "img")
                    except NoSuchElementException:
                        continue

                    total += 1

                    # Collect the product details
                    product_ids.append(f"{''.join(list(x[0] for x in search_term.split()))}_{'{0:0>10}'.format(str(total))}")
                    product_urls.append(product.find_element(By.TAG_NAME, "a").get_property("href"))
                    image_urls.append(image.get_property("src"))
                    descriptions.append(image.get_property("alt"))
                    brands.append(image.get_property("alt").split()[0])
                    genders.append(search_term.split()[0])
                    categories.append(search_term.split()[1])
                    occasions.append(search_term.split()[2])

                try:
                    # Find and click the next button to go to the next page
                    next_button = wd.find_element(By.CLASS_NAME, "pagination-next")
                    if next_button:
                        next_button.click()
                    else:
                        break
                except NoSuchElementException:
                    break

            except (WebDriverException, TimeoutException) as e:
                print(f"Error occurred: {e}")
                if retry_count < max_retries:
                    retry_count += 1
                    print(f"Retrying... ({retry_count}/{max_retries})")
                    wd.refresh()  # Refresh the page and retry
                    time.sleep(5)  # Add a delay before retrying
                else:
                    print(f"Exceeded maximum retries ({max_retries}). Moving to the next search term.")
                    break  # Exit the loop and move to the next search term

    except WebDriverException as e:
        print(f"Critical error occurred: {e}")
        break  # Exit the loop if a critical error occurs

# Close the WebDriver
wd.quit()


In [5]:
df = pd.DataFrame()

df['product_id'] = product_ids
df['product_url'] = product_urls
df['image_url'] = image_urls
df['description'] = descriptions
df['brand'] = brands
df['gender'] = genders
df['category'] = categories
df['type'] = occasions
df

,product_id,product_url,image_url,description,brand,gender,category,type
0,mes_0000000001,https://www.myntra.com/sunglasses/kaen+eyewear...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",KAEN EYEWEAR Unisex Square Sunglasses with Pol...,KAEN,men,eyewear,sunglasses
1,mes_0000000002,https://www.myntra.com/sunglasses/kaen+eyewear...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",KAEN EYEWEAR Unisex Square Sunglasses with Pol...,KAEN,men,eyewear,sunglasses
2,mes_0000000003,https://www.myntra.com/sunglasses/resist+eyewe...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",RESIST EYEWEAR Unisex Square Sunglasses with U...,RESIST,men,eyewear,sunglasses
3,mes_0000000004,https://www.myntra.com/sunglasses/kaen+eyewear...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",KAEN EYEWEAR Unisex Square Sunglasses with Pol...,KAEN,men,eyewear,sunglasses
4,mes_0000000005,https://www.myntra.com/sunglasses/kaen+eyewear...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",KAEN EYEWEAR Unisex Square Sunglasses with Pol...,KAEN,men,eyewear,sunglasses
...,...,...,...,...,...,...,...,...
2118,hhh_0000000702,https://www.myntra.com/handbags/anouk/anouk-bl...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",Anouk Blue Printed Structured Tote Bag with Cu...,Anouk,handbags,handbags,handbags
2119,hhh_0000000703,https://www.myntra.com/handbags/eske/eske-embe...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",Eske Embellished Bowling Handheld Bag with Tas...,Eske,handbags,handbags,handbags
2120,hhh_0000000704,https://www.myntra.com/handbags/baggit/baggit-...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",Baggit Women Printed Sling Bag,Baggit,handbags,handbags,handbags
2121,hhh_0000000705,https://www.myntra.com/handbags/forever+21/for...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",FOREVER 21 Textured Structured Shoulder Bag,FOREVER,handbags,handbags,handbags


In [6]:
#Lets findout if there are any duplicate rows
df.duplicated().sum()

0

In [7]:
are_duplicates = df['product_id'].duplicated().any()
are_duplicates

False

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2123 entries, 0 to 2122
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   product_id   2123 non-null   object
 1   product_url  2123 non-null   object
 2   image_url    2123 non-null   object
 3   description  2123 non-null   object
 4   brand        2123 non-null   object
 5   gender       2123 non-null   object
 6   category     2123 non-null   object
 7   type         2123 non-null   object
dtypes: object(8)
memory usage: 132.8+ KB


In [12]:
df["gender"].unique()

array(['men', 'women', 'handbags'], dtype=object)

In [15]:
#Now lets remove handbags from gender and replace it with Both(because it has products for both gender)
df['gender'] = df['gender'].replace('handbags', 'Both')
df["gender"].unique()

array(['men', 'women', 'Both'], dtype=object)

In [13]:
df["type"].unique()

array(['sunglasses', 'handbags'], dtype=object)

In [14]:
df["category"].unique()

array(['eyewear', 'handbags'], dtype=object)

In [18]:
#Lets load our df which we have scrapped and stored earlier. And then combine this df with the above df that we got here
df2 = pd.read_csv("./Scrapped_dataframe/scraped-myntra-fashion-products.csv")
df2


,product_id,product_url,image_url,description,brand,gender,category,type
0,mtf_0000000001,https://www.myntra.com/blazers/manq/manq-men-b...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",MANQ Men Black Slim Fit Solid Single Breasted ...,MANQ,men,topwear,formal
1,mtf_0000000002,https://www.myntra.com/blazers/peter+england+e...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",Peter England Elite Self-Design Single-Breaste...,Peter,men,topwear,formal
2,mtf_0000000003,https://www.myntra.com/blazers/louis+philippe/...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",Louis Philippe Textured Self Design Slim Fit S...,Louis,men,topwear,formal
3,mtf_0000000004,https://www.myntra.com/blazers/the+indian+gara...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",The Indian Garage Co Men Grey Solid Single-Bre...,The,men,topwear,formal
4,mtf_0000000005,https://www.myntra.com/blazers/peter+england+e...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",Peter England Elite Men Navy Blue Solid Slim-F...,Peter,men,topwear,formal
...,...,...,...,...,...,...,...,...
14659,wfs_0000000601,https://www.myntra.com/sports-shoes/tpent/tpen...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",TPENT Women Super Grip EVA Mesh Running Shoes,TPENT,women,footwear,sports
14660,wfs_0000000602,https://www.myntra.com/sports-shoes/tpent/tpen...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",TPENT Women Super Grip EVA Mesh Running Shoes,TPENT,women,footwear,sports
14661,wfs_0000000603,https://www.myntra.com/sports-shoes/tpent/tpen...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",TPENT Women Super Grip EVA Mesh Running Shoes,TPENT,women,footwear,sports
14662,wfs_0000000604,https://www.myntra.com/sports-shoes/tpent/tpen...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",TPENT Women Super Grip EVA Mesh Running Shoes,TPENT,women,footwear,sports


In [19]:
df2["category"].unique()

array(['topwear', 'bottomwear', 'one', 'footwear'], dtype=object)

In [20]:
#Lets replace "one" with "fullbodywear"
df2['category'] = df2['category'].replace('one', 'full_topwear')
df2["category"].unique()

array(['topwear', 'bottomwear', 'full_topwear', 'footwear'], dtype=object)

In [22]:
df2["type"].unique()

array(['formal', 'casual', 'party', 'ethnic', 'pyjama', 'piece', 'sports'],
      dtype=object)

In [23]:
#Lets replace "piece" with "onepiece"
df2['type'] = df2['type'].replace('piece', 'onepiece')
df2["type"].unique()

array(['formal', 'casual', 'party', 'ethnic', 'pyjama', 'onepiece',
       'sports'], dtype=object)

In [24]:
df2["gender"].unique()

array(['men', 'women'], dtype=object)

In [25]:
# Combine the DataFrames
final_df = pd.concat([df2, df], ignore_index=True)
final_df

,product_id,product_url,image_url,description,brand,gender,category,type
0,mtf_0000000001,https://www.myntra.com/blazers/manq/manq-men-b...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",MANQ Men Black Slim Fit Solid Single Breasted ...,MANQ,men,topwear,formal
1,mtf_0000000002,https://www.myntra.com/blazers/peter+england+e...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",Peter England Elite Self-Design Single-Breaste...,Peter,men,topwear,formal
2,mtf_0000000003,https://www.myntra.com/blazers/louis+philippe/...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",Louis Philippe Textured Self Design Slim Fit S...,Louis,men,topwear,formal
3,mtf_0000000004,https://www.myntra.com/blazers/the+indian+gara...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",The Indian Garage Co Men Grey Solid Single-Bre...,The,men,topwear,formal
4,mtf_0000000005,https://www.myntra.com/blazers/peter+england+e...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",Peter England Elite Men Navy Blue Solid Slim-F...,Peter,men,topwear,formal
...,...,...,...,...,...,...,...,...
16782,hhh_0000000702,https://www.myntra.com/handbags/anouk/anouk-bl...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",Anouk Blue Printed Structured Tote Bag with Cu...,Anouk,Both,handbags,handbags
16783,hhh_0000000703,https://www.myntra.com/handbags/eske/eske-embe...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",Eske Embellished Bowling Handheld Bag with Tas...,Eske,Both,handbags,handbags
16784,hhh_0000000704,https://www.myntra.com/handbags/baggit/baggit-...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",Baggit Women Printed Sling Bag,Baggit,Both,handbags,handbags
16785,hhh_0000000705,https://www.myntra.com/handbags/forever+21/for...,"https://assets.myntassets.com/dpr_2,q_60,w_210...",FOREVER 21 Textured Structured Shoulder Bag,FOREVER,Both,handbags,handbags


In [27]:
#Save this above dataframe into my local as a csv file
final_df.to_csv("./Scrapped_dataframe/final-scraped-myntra-fashion-products.csv", index=False)